In [ ]:
import joblib
import numpy as np

# Load the trained model
model_path = 'spam_classifier_model.pkl'
model = joblib.load(model_path)

# Predict function
def predict_spam(text):
    proba = model.predict_proba([text])[0]
    spam_proba = proba[1] * 100
    not_spam_proba = proba[0] * 100
    if spam_proba > not_spam_proba:
        return f'This text is {spam_proba:.2f}% likely to be Spam.'
    else:
        return f'This text is {not_spam_proba:.2f}% likely to be Not Spam.'

# Example usage
if __name__ == "__main__":
    example_text = """Reminder: You have one day left (until 01/07) to reset your Apple Account password.
To complete this, go to www.apple.com/recover"""
    print(predict_spam(example_text))

This text is 63.50% likely to be Not Spam.
